<a href="https://colab.research.google.com/github/moulikatou/Ter_securite_Alimentaire_Miashs/blob/vmelancon/rCSI_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# rCSI Prediction

In this notebook we'll try to train a model to try and predict the rCSI index in Burkina Faso. We will first gather some features to fit the model on, then we'll see how those features could explain an rCSI variability in time and in space. So let's begin !

## Imports / Global Variables

In [1]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive 
drive.mount("/content/gdrive")

PATH = "/content/gdrive/MyDrive/COLAB_STUFF/donnees/"
y_filename = "reduced_named_data.csv"
X_filename = ["matriceXY_Precipitation.csv", "MatriceXY_dataMais.csv"]

Mounted at /content/gdrive


# Data Gathering

The first thing to do will be to load data and merge the differents DataFrames.

In [2]:
y_df = pd.read_csv(PATH + y_filename).iloc[:, 2:]
cols = ["VIL", "MEN", "YEAR"]
y_df[cols] = y_df[cols].astype(int)
y_df.head()

,PROV,COM,VIL,MEN,YEAR,Q1,Q2,Q3,Q4,Q5,CSI
0,NAMENTENGA,BOALA,1,45,2017,0.0,0.0,0.0,0.0,0.0,0.0
1,NAMENTENGA,BOALA,1,58,2017,3.0,2.0,4.0,2.0,0.0,17.0
2,NAMENTENGA,BOALA,1,67,2017,0.0,0.0,0.0,0.0,0.0,0.0
3,NAMENTENGA,BOALA,1,75,2017,0.0,0.0,0.0,0.0,0.0,0.0
4,NAMENTENGA,BOALA,1,84,2017,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
X_dfs = [pd.read_csv(PATH + filename).iloc[:, 1:] for filename in X_filename]

In [4]:
precipitations = X_dfs[0]
precipitations.head()

,ADM1_FR,ADM1_PCODE,ADM2_FR,ADM2_PCODE,Preci_janvier_mean,Preci_janvier_min,Preci_janvier_max,Preci_fevrier_mean,Preci_fevrier_min,Preci_fevrier_max,Preci_mars_mean,Preci_mars_min,Preci_mars_max,Preci_avril_mean,Preci_avril_min,Preci_avril_max,Preci_mai_mean,Preci_mai_min,Preci_mai_max,Preci_juin_mean,Preci_juin_min,Preci_juin_max,Preci_juillet_mean,Preci_juillet_min,Preci_juillet_max,Preci_aout_mean,Preci_aout_min,Preci_aout_max,Preci_septembre_mean,Preci_septembre_min,Preci_septembre_max,Preci_octobre_mean,Preci_octobre_min,Preci_octobre_max,Preci_novembre_mean,Preci_novembre_min,Preci_novembre_max,Preci_decembre_mean,Preci_decembre_min,Preci_decembre_max,Année,RcsiMoyProv
0,Boucle du Mouhoun,BF46,Balé,BF4601,0.895123,0.000000,1.040231,1.001181,0.616728,1.343220,9.523333,5.957644,17.382250,48.796281,37.748707,58.656685,109.199602,97.480064,119.457535,167.725002,156.914047,181.091324,252.297996,246.928986,262.001221,296.369484,282.773560,307.212494,155.322938,138.880219,168.172714,15.920349,14.383471,17.770554,3.789285,2.456811,5.557866,0.568468,0.353002,0.828910,2014,8.396954
1,Centre-Nord,BF49,Bam,BF4901,0.109375,0.000000,1.000000,0.223958,0.000000,1.000000,2.333921,1.746255,4.111912,19.891068,11.262523,27.773399,50.791151,36.893196,66.137444,127.598660,99.636345,141.369675,217.486235,187.093918,244.460434,239.527905,215.697510,252.526215,101.301880,88.960663,114.385323,12.949035,11.263667,14.564628,0.570640,0.000000,1.000000,1.000000,1.000000,1.000000,2014,3.293335
2,Boucle du Mouhoun,BF46,Banwa,BF4602,0.339588,0.000000,1.488208,1.461171,0.521126,1.949252,3.997883,2.571072,5.747647,29.311306,24.941317,34.665615,90.487470,82.896545,97.319786,175.530994,160.391861,183.575455,266.221875,246.155380,277.838867,276.688683,268.699005,295.294159,143.777325,123.788307,158.052704,21.455945,16.433945,26.704906,4.764787,2.177440,6.542110,1.509296,0.667497,2.314100,2014,2.754675
3,Centre-Sud,BF51,Bazèga,BF5101,0.000000,0.000000,0.000000,0.907997,0.700010,1.680727,9.450436,6.612926,14.617943,43.354781,35.308796,50.593292,94.079683,84.913246,102.673264,144.472180,135.819824,155.804947,260.421237,246.157242,274.659363,274.778534,265.286926,279.767303,130.207311,117.140663,141.587112,13.244244,12.238208,14.849860,1.190055,0.550620,1.765850,0.987993,0.630210,1.345117,2014,5.822797
4,Sud-Ouest,BF57,Bougouriba,BF5701,2.717166,1.709912,3.627472,1.697388,1.047621,2.561265,22.448071,16.613667,27.242189,53.546841,47.277435,57.877853,117.436442,108.583351,123.723640,165.165251,154.163330,170.622406,249.045711,241.879456,254.171265,284.064806,266.193268,300.148499,183.521020,175.643539,192.507919,21.996138,18.423113,28.447493,9.095707,5.485919,14.202320,0.777160,0.341059,1.064903,2014,8.841051


In [5]:
mais = X_dfs[1]
mais.head()

,REGION,PROVINCE,COMMUNE,ANNEE,Sonagess_01mais-(t-0),Sonagess_02mais-(t-0),Sonagess_03mais-(t-0),Sonagess_04mais-(t-0),Sonagess_05mais-(t-0),Sonagess_06mais-(t-0),Sonagess_07mais-(t-0),Sonagess_08mais-(t-0),Sonagess_09mais-(t-0),Sonagess_10mais-(t-0),Sonagess_11mais-(t-0),Sonagess_12mais-(t-0),RcsiMoyProv
0,BOUCLE DU MOUHOUN,BALE,BAGASSI,2014,104.840726,108.256734,99.388983,104.830742,99.085454,94.798132,96.862475,97.653481,104.737791,92.988742,86.203519,94.920836,8.396954
1,BOUCLE DU MOUHOUN,BALE,BANA,2014,112.479671,111.664600,106.305087,113.098474,108.079178,106.271666,107.724837,111.586062,114.685928,100.033389,93.315756,99.961771,8.396954
2,BOUCLE DU MOUHOUN,BALE,BOROMO,2014,106.788935,109.454099,102.288154,106.050591,106.158558,101.277340,102.339825,100.441724,106.443285,91.906695,86.523674,94.649666,8.396954
3,BOUCLE DU MOUHOUN,BALE,FARA,2014,99.487754,101.147545,94.462890,98.755899,102.045788,98.264281,98.850299,93.851711,101.754563,86.974468,83.265693,89.510758,8.396954
4,BOUCLE DU MOUHOUN,BALE,OURY,2014,110.594379,113.225762,106.901253,110.234317,109.679245,105.012713,105.998145,104.950970,109.337222,94.972882,89.658260,98.584088,8.396954


Now we format those dataframes to be merged together.

In [9]:
try:
    del precipitations["RcsiMoyProv"]
except:
    pass
precipitations = precipitations.rename(columns={"ADM1_FR" : "PROV", "ADM2_FR" : "COM", "Année" : "ANNEE"})
precipitations["PROV"] = precipitations["PROV"].apply(lambda x : x.upper())

precipitations.to_csv(PATH + "precipitations.csv")
precipitations.head()

,PROV,ADM1_PCODE,COM,ADM2_PCODE,Preci_janvier_mean,Preci_janvier_min,Preci_janvier_max,Preci_fevrier_mean,Preci_fevrier_min,Preci_fevrier_max,Preci_mars_mean,Preci_mars_min,Preci_mars_max,Preci_avril_mean,Preci_avril_min,Preci_avril_max,Preci_mai_mean,Preci_mai_min,Preci_mai_max,Preci_juin_mean,Preci_juin_min,Preci_juin_max,Preci_juillet_mean,Preci_juillet_min,Preci_juillet_max,Preci_aout_mean,Preci_aout_min,Preci_aout_max,Preci_septembre_mean,Preci_septembre_min,Preci_septembre_max,Preci_octobre_mean,Preci_octobre_min,Preci_octobre_max,Preci_novembre_mean,Preci_novembre_min,Preci_novembre_max,Preci_decembre_mean,Preci_decembre_min,Preci_decembre_max,ANNEE
0,BOUCLE DU MOUHOUN,BF46,Balé,BF4601,0.895123,0.000000,1.040231,1.001181,0.616728,1.343220,9.523333,5.957644,17.382250,48.796281,37.748707,58.656685,109.199602,97.480064,119.457535,167.725002,156.914047,181.091324,252.297996,246.928986,262.001221,296.369484,282.773560,307.212494,155.322938,138.880219,168.172714,15.920349,14.383471,17.770554,3.789285,2.456811,5.557866,0.568468,0.353002,0.828910,2014
1,CENTRE-NORD,BF49,Bam,BF4901,0.109375,0.000000,1.000000,0.223958,0.000000,1.000000,2.333921,1.746255,4.111912,19.891068,11.262523,27.773399,50.791151,36.893196,66.137444,127.598660,99.636345,141.369675,217.486235,187.093918,244.460434,239.527905,215.697510,252.526215,101.301880,88.960663,114.385323,12.949035,11.263667,14.564628,0.570640,0.000000,1.000000,1.000000,1.000000,1.000000,2014
2,BOUCLE DU MOUHOUN,BF46,Banwa,BF4602,0.339588,0.000000,1.488208,1.461171,0.521126,1.949252,3.997883,2.571072,5.747647,29.311306,24.941317,34.665615,90.487470,82.896545,97.319786,175.530994,160.391861,183.575455,266.221875,246.155380,277.838867,276.688683,268.699005,295.294159,143.777325,123.788307,158.052704,21.455945,16.433945,26.704906,4.764787,2.177440,6.542110,1.509296,0.667497,2.314100,2014
3,CENTRE-SUD,BF51,Bazèga,BF5101,0.000000,0.000000,0.000000,0.907997,0.700010,1.680727,9.450436,6.612926,14.617943,43.354781,35.308796,50.593292,94.079683,84.913246,102.673264,144.472180,135.819824,155.804947,260.421237,246.157242,274.659363,274.778534,265.286926,279.767303,130.207311,117.140663,141.587112,13.244244,12.238208,14.849860,1.190055,0.550620,1.765850,0.987993,0.630210,1.345117,2014
4,SUD-OUEST,BF57,Bougouriba,BF5701,2.717166,1.709912,3.627472,1.697388,1.047621,2.561265,22.448071,16.613667,27.242189,53.546841,47.277435,57.877853,117.436442,108.583351,123.723640,165.165251,154.163330,170.622406,249.045711,241.879456,254.171265,284.064806,266.193268,300.148499,183.521020,175.643539,192.507919,21.996138,18.423113,28.447493,9.095707,5.485919,14.202320,0.777160,0.341059,1.064903,2014


In [10]:
try:
    del mais["RcsiMoyProv"]
except:
    pass
mais = mais.rename(columns={"PROVINCE" : "PROV", "COMMUNE" : "COM"})
mais.head()

,REGION,PROV,COM,ANNEE,Sonagess_01mais-(t-0),Sonagess_02mais-(t-0),Sonagess_03mais-(t-0),Sonagess_04mais-(t-0),Sonagess_05mais-(t-0),Sonagess_06mais-(t-0),Sonagess_07mais-(t-0),Sonagess_08mais-(t-0),Sonagess_09mais-(t-0),Sonagess_10mais-(t-0),Sonagess_11mais-(t-0),Sonagess_12mais-(t-0)
0,BOUCLE DU MOUHOUN,BALE,BAGASSI,2014,104.840726,108.256734,99.388983,104.830742,99.085454,94.798132,96.862475,97.653481,104.737791,92.988742,86.203519,94.920836
1,BOUCLE DU MOUHOUN,BALE,BANA,2014,112.479671,111.664600,106.305087,113.098474,108.079178,106.271666,107.724837,111.586062,114.685928,100.033389,93.315756,99.961771
2,BOUCLE DU MOUHOUN,BALE,BOROMO,2014,106.788935,109.454099,102.288154,106.050591,106.158558,101.277340,102.339825,100.441724,106.443285,91.906695,86.523674,94.649666
3,BOUCLE DU MOUHOUN,BALE,FARA,2014,99.487754,101.147545,94.462890,98.755899,102.045788,98.264281,98.850299,93.851711,101.754563,86.974468,83.265693,89.510758
4,BOUCLE DU MOUHOUN,BALE,OURY,2014,110.594379,113.225762,106.901253,110.234317,109.679245,105.012713,105.998145,104.950970,109.337222,94.972882,89.658260,98.584088


In [14]:
X = pd.concat([mais, precipitations], axis=0)
X

,REGION,PROV,COM,ANNEE,Sonagess_01mais-(t-0),Sonagess_02mais-(t-0),Sonagess_03mais-(t-0),Sonagess_04mais-(t-0),Sonagess_05mais-(t-0),Sonagess_06mais-(t-0),Sonagess_07mais-(t-0),Sonagess_08mais-(t-0),Sonagess_09mais-(t-0),Sonagess_10mais-(t-0),Sonagess_11mais-(t-0),Sonagess_12mais-(t-0),ADM1_PCODE,ADM2_PCODE,Preci_janvier_mean,Preci_janvier_min,Preci_janvier_max,Preci_fevrier_mean,Preci_fevrier_min,Preci_fevrier_max,Preci_mars_mean,Preci_mars_min,Preci_mars_max,Preci_avril_mean,Preci_avril_min,Preci_avril_max,Preci_mai_mean,Preci_mai_min,Preci_mai_max,Preci_juin_mean,Preci_juin_min,Preci_juin_max,Preci_juillet_mean,Preci_juillet_min,Preci_juillet_max,Preci_aout_mean,Preci_aout_min,Preci_aout_max,Preci_septembre_mean,Preci_septembre_min,Preci_septembre_max,Preci_octobre_mean,Preci_octobre_min,Preci_octobre_max,Preci_novembre_mean,Preci_novembre_min,Preci_novembre_max,Preci_decembre_mean,Preci_decembre_min,Preci_decembre_max
0,BOUCLE DU MOUHOUN,BALE,BAGASSI,2014,104.840726,108.256734,99.388983,104.830742,99.085454,94.798132,96.862475,97.653481,104.737791,92.988742,86.203519,94.920836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BOUCLE DU MOUHOUN,BALE,BANA,2014,112.479671,111.664600,106.305087,113.098474,108.079178,106.271666,107.724837,111.586062,114.685928,100.033389,93.315756,99.961771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BOUCLE DU MOUHOUN,BALE,BOROMO,2014,106.788935,109.454099,102.288154,106.050591,106.158558,101.277340,102.339825,100.441724,106.443285,91.906695,86.523674,94.649666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BOUCLE DU MOUHOUN,BALE,FARA,2014,99.487754,101.147545,94.462890,98.755899,102.045788,98.264281,98.850299,93.851711,101.754563,86.974468,83.265693,89.510758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BOUCLE DU MOUHOUN,BALE,OURY,2014,110.594379,113.225762,106.901253,110.234317,109.679245,105.012713,105.998145,104.950970,109.337222,94.972882,89.658260,98.584088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,NaN,SAHEL,Yagha,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BF56,BF5604,0.000000,0.0,0.000000,0.114007,0.000000,1.000000,2.472657,1.723278,3.272525,12.756782,6.703487,21.748665,31.651584,22.278521,40.927975,103.632518,88.321281,115.141457,155.466487,138.836716,169.372131,206.956324,190.885788,221.006393,131.667950,115.263069,147.500931,8.216859,6.067113,10.412515,0.944625,0.000000,1.000000,0.000000,0.000000,0.00000
221,NaN,NORD,Yatenga,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BF54,BF5403,0.000000,0.0,0.000000,0.012292,0.000000,0.989102,1.807885,0.842420,3.789382,14.472554,8.318072,24.641951,48.152401,35.088348,69.584290,115.262039,93.140549,139.460785,216.067661,186.796738,250.309341,223.697138,203.304077,248.460129,87.955605,75.205811,102.253967,14.821808,11.880961,17.107590,0.546515,0.000000,1.000000,0.428125,0.000000,1.00000
222,NaN,CENTRE-OUEST,Ziro,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BF50,BF5004,0.011501,0.0,0.965833,1.302805,0.752650,1.686769,13.787027,9.460610,19.486679,52.027115,43.271828,60.887390,102.514971,96.886330,115.530197,151.041181,144.226288,156.078354,264.521778,254.044113,280.016266,283.598749,271.815521,297.254486,146.275954,135.901749,153.461182,13.343771,11.786141,16.446672,2.110892,1.